In [1]:
!nvidia-smi

Thu Oct 31 11:55:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install accelerate
!pip install transformers==4.45.2
!pip install bitsandbytes
!pip install datasets
!pip install rouge-score
!pip install pymorphy2
!pip install peft
#!pip install flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
  Preparing me

In [3]:
import random
from typing import List, Dict

import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
from tqdm import tqdm


class ChatDataset(Dataset):
    def __init__(
        self,
        original_records: List[Dict],
        tokenizer: AutoTokenizer,
        max_tokens_count: int,
        sample_rate: float = 1.0,
        only_target_loss: bool = True,
        add_global_bos: bool = True,
        add_global_eos: bool = True,
        labels_pad_token_id: int = -100,
    ):
        self.original_records = original_records
        self.sample_rate = sample_rate
        self.tokenizer = tokenizer
        self.max_tokens_count = max_tokens_count
        self.only_target_loss = only_target_loss
        self.labels_pad_token_id = labels_pad_token_id
        self.add_global_bos = add_global_bos
        self.add_global_eos = add_global_eos
        self.is_printed = False

        self.records = []
        for record in tqdm(original_records):
            if random.random() > self.sample_rate:
                continue
            tensors = self.convert_record(record)
            if tensors is None:
                continue
            self.records.append(tensors)

    def __len__(self):
        return len(self.records)

    def __getitem__(self, index):
        return self.records[index]

    def get_tokens(self, messages):
        #print(messages)
        tokens = self.tokenizer.apply_chat_template(
            messages,
            add_special_tokens=False,
            tokenize=True,
            add_generation_prompt=False,
        )
        if tokens[0] == self.tokenizer.bos_token_id:
            tokens = tokens[1:]
        return tokens

    def convert_record(self, record):
        input_ids, labels = [], []

        for i, message in enumerate(record["messages"]):
            if message['role'] == 'bot':
                message['role'] = 'assistant'
                record["messages"][i]['role'] = 'assistant'

            message_input_ids = self.get_tokens([message])
            message_labels = message_input_ids
            if len(input_ids) + len(message_input_ids) > self.max_tokens_count - 2:
                break

            labels_mask = [
                self.labels_pad_token_id for _ in range(len(message_input_ids))
            ]
            if (
                message["role"] not in ("assistant", "bot", "gpt")
                and self.only_target_loss
            ):
                message_labels = labels_mask

            input_ids.extend(message_input_ids)
            labels.extend(message_labels)

        if not input_ids:
            return None

        original_input_ids = self.get_tokens(record["messages"])
        if input_ids != original_input_ids[: len(input_ids)]:
            print(input_ids)
            print(original_input_ids[: len(input_ids)])
        assert input_ids == original_input_ids[: len(input_ids)]

        if self.add_global_bos and input_ids[0] != self.tokenizer.bos_token_id:
            input_ids.insert(0, self.tokenizer.bos_token_id)
            labels.insert(0, self.labels_pad_token_id)

        if input_ids[-2] == self.tokenizer.eos_token_id:
            input_ids = input_ids[:-1]
            labels = labels[:-1]

        if self.add_global_eos and input_ids[-1] != self.tokenizer.eos_token_id:
            input_ids.append(self.tokenizer.eos_token_id)
            labels.append(self.tokenizer.eos_token_id)

        if not self.is_printed:
            print(input_ids)
            print(labels)
            print(
                "Full prompt:" +
                self.tokenizer.decode(input_ids, skip_special_tokens=False)
            )
            assert '\n' in self.tokenizer.decode(input_ids, skip_special_tokens=False)
            self.is_printed = True

        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(labels)
        attention_mask = input_ids.new_ones(input_ids.size())
        assert (
            input_ids.size(0)
            == labels.size(0)
            == attention_mask.size(0)
            <= self.max_tokens_count
        )
        return {
            "input_ids": input_ids,
            "labels": labels,
            "attention_mask": attention_mask,
        }

In [15]:
from datasets import load_dataset
dataset = load_dataset('IlyaGusev/saiga_scored')
dataset = dataset['train'].select(range(1000))
dataset = dataset.train_test_split(test_size=0.1)
dataset

DatasetDict({
    train: Dataset({
        features: ['messages', 'source', 'opus_score', 'language', 'turns', 'sonnet_topic', 'sonnet_topic_explanation', 'sonnet_complexity', 'sonnet_complexity_explanation', 'is_bad_by_regex', 'score_explanation'],
        num_rows: 900
    })
    test: Dataset({
        features: ['messages', 'source', 'opus_score', 'language', 'turns', 'sonnet_topic', 'sonnet_topic_explanation', 'sonnet_complexity', 'sonnet_complexity_explanation', 'is_bad_by_regex', 'score_explanation'],
        num_rows: 100
    })
})

In [6]:
import random
import json
import os

import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForTokenClassification,
    AutoConfig,
)
from transformers import (
    Trainer,
    TrainingArguments,
    logging,
    TrainerCallback,
    TrainerState,
    TrainerControl,
    BitsAndBytesConfig,
)
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig
import re
from peft import prepare_model_for_kbit_training
import codecs

os.environ["WANDB_DISABLED"] = "true"

In [7]:
instruct_model_name = 'RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4'

tokenizer = AutoTokenizer.from_pretrained(instruct_model_name)
chat_template = tokenizer.chat_template
bos_token = tokenizer.bos_token
eos_token = tokenizer.eos_token
pad_token = tokenizer.pad_token
chat_template

tokenizer_config.json:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/3.37M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/2.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/12.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/759 [00:00<?, ?B/s]

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- messages[0][\'content\'] }}\n    {%- else %}\n        {{- \'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.\' }}\n    {%- endif %}\n    {{- "\\n\\n# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0][\'role\'] == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0][\'content\'] + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- en

In [9]:
model_name = 'RefalMachine/ruadapt_qwen2.5_3B_u48_full_lr3e4_bs256'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.bos_token = bos_token
tokenizer.eos_token = eos_token
tokenizer.pad_token = pad_token
tokenizer.chat_template = chat_template
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.41M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

In [16]:
only_target_loss = True
max_tokens_count = 512
datasets = []
for records in (dataset['train'], dataset['test']):
    datasets.append(
        ChatDataset(
            records,
            tokenizer,
            max_tokens_count=max_tokens_count,
            sample_rate=1.0,
            only_target_loss=only_target_loss,
            add_global_eos=False,
            add_global_bos=False
        )
    )
train_dataset, val_dataset = datasets

  3%|▎         | 29/900 [00:00<00:05, 147.16it/s]

[48001, 14598, 13, 12647, 1883, 1641, 40092, 277, 4342, 593, 367, 293, 1818, 4098, 337, 260, 48002, 261, 13, 48001, 17130, 13, 885, 2676, 28812, 405, 20669, 456, 259, 327, 272, 813, 641, 354, 1336, 2164, 259, 905, 970, 1918, 2002, 276, 5071, 367, 259, 29049, 367, 259, 3640, 259, 9205, 263, 22961, 276, 17161, 879, 296, 3555, 263, 695, 384, 4651, 354, 11425, 2344, 20669, 456, 260, 48002, 261, 13]
[-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 48001, 17130, 13, 885, 2676, 28812, 405, 20669, 456, 259, 327, 272, 813, 641, 354, 1336, 2164, 259, 905, 970, 1918, 2002, 276, 5071, 367, 259, 29049, 367, 259, 3640, 259, 9205, 263, 22961, 276, 17161, 879, 296, 3555, 263, 695, 384, 4651, 354, 11425, 2344, 20669, 456, 260, 48002, 261, 13]
Full prompt:<|im_start|> user
Перескажи сказку "Репка" двумя предложениями.<|im_end|> 
<|im_start|> assistant
Дед вырастил репку, но не мог её выдернуть, поэтому каждый член семьи - бабка, внучка, к

 41%|████      | 41/100 [00:00<00:00, 206.95it/s]

[48001, 14598, 13, 18771, 1708, 278, 10131, 313, 1842, 7053, 32928, 270, 2042, 300, 298, 633, 15574, 267, 970, 298, 1305, 22259, 282, 5248, 5578, 23635, 259, 14836, 17191, 263, 40262, 358, 48002, 261, 13, 48001, 17130, 13, 616, 7138, 383, 23635, 287, 289, 13, 264, 260, 25140, 265, 39783, 298, 36402, 287, 270, 1488, 290, 274, 442, 478, 263, 3169, 2444, 325, 260, 13, 271, 260, 6444, 286, 1159, 20090, 270, 18301, 345, 259, 34811, 300, 263, 4654, 956, 38154, 325, 260, 13, 13, 13706, 338, 16956, 289, 13, 264, 260, 6411, 274, 45222, 319, 347, 14316, 300, 261, 1366, 14635, 270, 12733, 345, 263, 18373, 389, 261, 5739, 2341, 300, 260, 13, 271, 260, 9167, 6591, 298, 261, 2635, 3443, 287, 270, 14316, 300, 765, 337, 16792, 373, 263, 17849, 389, 40496, 270, 487, 4868, 29331, 1615, 260, 13, 13, 10741, 35793, 287, 289, 13, 264, 260, 872, 9316, 2203, 339, 8649, 490, 270, 14843, 1780, 389, 261, 14902, 482, 263, 34146, 278, 18274, 345, 260, 13, 271, 260, 18299, 345, 1306, 270, 471, 2933, 300, 311, 3230,

100%|██████████| 100/100 [00:00<00:00, 263.77it/s]


In [17]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=f"cuda:0",
    torch_dtype=torch.float16,
    attn_implementation="sdpa",
)
prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/752M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(48022, 2048, padding_idx=48000)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear4bit(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm

In [20]:
messages = [{'role': 'user', 'content': 'Напиши что такое LLM.'}]
tokenizer.apply_chat_template(messages, return_tensors='pt', add_special_tokens=True, add_generation_prompt=True)

tensor([[48001, 14598,    13,  1410, 37762,   279,  1001,   981, 35642,   260,
         48002,   261,    13, 48001, 17130,    13]])

In [21]:
from transformers import GenerationConfig

def generate(messages, model, tokenizer, generation_config):
    input_ids = tokenizer.apply_chat_template(messages, return_tensors='pt', add_special_tokens=True, add_generation_prompt=True)
    input_ids = input_ids.to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            generation_config=generation_config
        )
    outputs = []
    for sample_output_ids, sample_input_ids in zip(output_ids, input_ids):
        sample_output_ids = sample_output_ids[len(sample_input_ids):]
        sample_output = tokenizer.decode(sample_output_ids, skip_special_tokens=True)
        outputs.append(sample_output)

    if len(outputs) == 1:
        outputs = outputs[0]
    return outputs



generation_config = GenerationConfig.from_dict(
    {
        'top_k': 40,
        'top_p': 0.9,
        'temperature': 0.2,
        'repetition_penalty': 1.0,
        'max_new_tokens': 64,
        'do_sample': True,
        'pad_token_id': tokenizer.pad_token_id,
        'bos_token_id': tokenizer.bos_token_id,
        'eos_token_id': tokenizer.eos_token_id
    }
)
generation_config

GenerationConfig {
  "bos_token_id": 48000,
  "do_sample": true,
  "eos_token_id": 48002,
  "max_new_tokens": 64,
  "pad_token_id": 48000,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

In [22]:
generate(messages, model, tokenizer, generation_config)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


'LLM (Larynx-Laryngium-Microphone) is a type haystack of artificial intelligence that is designed to understand and generate human language. It is trained on large amounts of text data and can perform a variety of natural language processing tasks, such as language translation,'

In [24]:
lora_config = {
    "r": 16,
    "lora_alpha": 16,
    "lora_dropout": 0.0,
    "bias": "none",
    "target_modules": ["q_proj", "v_proj", "k_proj", "o_proj"]
}
lora_config = LoraConfig(**lora_config)
lora_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=16, target_modules={'q_proj', 'o_proj', 'k_proj', 'v_proj'}, lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [25]:
lora_config.modules_to_save

In [28]:
model = get_peft_model(model, lora_config)
if model.config.tie_word_embeddings and lora_config.modules_to_save is not None and 'lm_head' in lora_config.modules_to_save:
    print('Tie embeddings')
    assert 'embed_tokens' not in lora_config.modules_to_save
    model.base_model.model.model.embed_tokens.weight = model.base_model.model.lm_head.modules_to_save["default"].weight

In [31]:
training_args = {
    "evaluation_strategy": "steps",
    "per_device_train_batch_size": 1,
    "per_device_eval_batch_size": 1,
    "gradient_accumulation_steps": 8,
    "eval_steps": 16,
    "save_steps": 128,
    "logging_steps": 1,
    "learning_rate": 0.00005,
    "num_train_epochs": 1,
    "lr_scheduler_type": "cosine",
    "warmup_steps": 16,
    "bf16": False,
    "fp16": True,
    "optim": "paged_adamw_8bit",
    "save_total_limit": 1,
    "seed": 1337,
    "max_grad_norm": 1.0,
    "weight_decay": 0.05
}
training_args = TrainingArguments(output_dir='./instruct', **training_args)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [32]:
data_collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=8)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)
if len(trainer.label_names) == 0:
    trainer.label_names.append('labels')

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [33]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss
16,3.747400,nan
32,2.204400,nan
48,2.459000,nan
64,7.685800,nan
80,3.051900,nan
96,3.027500,nan


TrainOutput(global_step=109, training_loss=4.810051315421358, metrics={'train_runtime': 963.2865, 'train_samples_per_second': 0.909, 'train_steps_per_second': 0.113, 'total_flos': 3382511530475520.0, 'train_loss': 4.810051315421358, 'epoch': 0.9954337899543378})

In [34]:
generate(messages, model, tokenizer, generation_config)

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'LLM (large language model) — это тип искусственного интеллекта, который способен обрабатывать и генерировать текст на основе больших объемов данных. LLMы обычно используются для задач, таких как перевод, текст-текст, текст-сводка и другие. Они обычно обучены на больших данных и могут генерировать текст, который может'